## Imports

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain import hub
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from pydantic.v1 import BaseModel, Field
from langchain.agents import AgentExecutor
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function

## Define Langchain Utilitites

In [3]:
class EmailInput(BaseModel):
    message: str = Field(..., description="The message content.")

@tool(args_schema=EmailInput, return_direct=True)
def register_send_email(message: str) -> str:
    """
    Send a message to Nikhil
    """
    print(f"TODO: Register message: {message}")
    return f"Your message has been registered for sending."

## Chatbot

In [4]:
EMBEDDING_FUNCTION = OpenAIEmbeddings(model=os.getenv("EMBEDDING_TYPE"))
EMBEDDING_FUNCTION

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1054b3070>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x108d95e40>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [5]:
PINECONE_VS = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=EMBEDDING_FUNCTION)
PINECONE_VS

In [6]:
search_kwargs_vs = {
    "k": int(os.getenv("TOP_K")),
}
if os.getenv("SEARCH_TYPE") == "mmr":
    search_kwargs_vs["fetch_k"] = int(os.getenv("FETCH_K"))
    search_kwargs_vs["lambda_multiplier"] = os.getenv("LAMBDA_MULTIPLIER")
RETRIEVER = PINECONE_VS.as_retriever(search_type=os.getenv("SEARCH_TYPE"), search_kwargs=search_kwargs_vs)
RETRIEVER

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x108dc0130>, search_kwargs={'k': 4})

In [7]:
PROMPT = hub.pull(os.getenv("LLM_RAG_PROMPT_NAME"))
PROMPT

ChatPromptTemplate(input_variables=['agent_scratchpad', 'context', 'question'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'portfolio-rag-prompt', 'lc_hub_commit_hash': '6ce75a46488dcbc416418fd15d6ad2c740d1c8c66a8b0b634b347c634a891353'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='You are an assistant for question-answering tasks and your name is Harpy. Always answer questions as a third person. Use the following pieces of retrieved context to answer the question. If the question is directly directed to you, for example, if the questions are "How are you?", "Who are you?", "Tell me about yourself.", etc.  just s

In [8]:
TOOLS = [register_send_email]
FUNCTIONS = [convert_to_openai_function(t) for t in TOOLS]
FUNCTIONS

[{'name': 'register_send_email',
  'description': 'Send a message to Nikhil',
  'parameters': {'type': 'object',
   'properties': {'message': {'description': 'The message content.',
     'type': 'string'}},
   'required': ['message']}}]

In [9]:
model_kwargs = {
    "top_p": float(os.getenv("LLM_TOP_P")),
    "frequency_penalty": float(os.getenv("LLM_FREQUENCY_PENALTY")),
    "presence_penalty": float(os.getenv("LLM_PRESENCE_PENALTY")),
}
LLM = ChatOpenAI(
    model=os.getenv("LLM_MODEL_NAME"),
    temperature=os.getenv("LLM_TEMPERATURE"),
    model_kwargs=model_kwargs
).bind(functions=FUNCTIONS)
LLM

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x108e74f10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x108e768c0>, temperature=1.0, model_kwargs={'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'functions': [{'name': 'register_send_email', 'description': 'Send a message to Nikhil', 'parameters': {'type': 'object', 'properties': {'message': {'description': 'The message content.', 'type': 'string'}}, 'required': ['message']}}]})

In [10]:
CHAIN = (
    RunnablePassthrough.assign(agent_scratchpad=(lambda x: format_to_openai_functions(x["intermediate_steps"])))
    | RunnablePassthrough.assign(context=(lambda x: x["question"]) | RETRIEVER | (lambda docs: "\n\n".join(doc.page_content for doc in docs)))
    | RunnablePassthrough.assign(question=(lambda x: x["question"]))
    | PROMPT
    | LLM
    | OpenAIFunctionsAgentOutputParser()
)
CHAIN

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_functions(x['intermediate_steps']))
})
| RunnableAssign(mapper={
    context: RunnableLambda(...)
             | VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x108dc0130>, search_kwargs={'k': 4})
             | RunnableLambda(...)
  })
| RunnableAssign(mapper={
    question: RunnableLambda(...)
  })
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'context', 'question'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'portfolio-rag-prompt', 'lc_hub_commit_hash': '6ce75a46488dcb

### Test 1

In [11]:
AGENT = AgentExecutor(agent=CHAIN, tools=TOOLS, max_iterations=os.getenv("LLM_AGENT_MAX_ITERATIONS"), verbose=True)
print('='*100)
print('='*100)
AGENT.invoke(
    {
        "question": "Hi"
    }
)
print('='*100)
print('='*100)
print("\n\n")
print('='*100)
print('='*100)
AGENT.invoke(
    {
        "question": "Send a message to nikhil asking for his indetail work experience?"
    }
)
print('='*100)
print('='*100)
print("\n\n")
print('='*100)
print('='*100)
AGENT.invoke(
    {
        "question": "Tell me about his work exp"
    }
)
print('='*100)
print('='*100)



> Entering new AgentExecutor chain...
Hey! I am Harpy, your chat assistant. Please ask questions about Nikhil. I can answer them for you :-)

> Finished chain.





> Entering new AgentExecutor chain...

Invoking: `register_send_email` with `{'message': 'Could you please provide me with more details about your work experience, especially focusing on the projects you have worked on and the roles you have undertaken in different organizations? Thank you!'}`


TODO: Register message: Could you please provide me with more details about your work experience, especially focusing on the projects you have worked on and the roles you have undertaken in different organizations? Thank you!
Your message has been registered for sending.


> Finished chain.





> Entering new AgentExecutor chain...
Nikhil has nearly six years of work experience in the computer science domain. He has worked as a CTO of a startup, a Machine Learning Engineer at Insureka, a research assistant, and a software enginee

### Test 2

In [12]:
AGENT = AgentExecutor(agent=CHAIN, tools=TOOLS, max_iterations=os.getenv("LLM_AGENT_MAX_ITERATIONS"), verbose=True)
AGENT.invoke(
    {
        "question": "Email, 'Hi how are you, I wanted to get in touch with you'"
    }
)



> Entering new AgentExecutor chain...

Invoking: `register_send_email` with `{'message': 'Hi, I wanted to get in touch with you. How are you?'}`


TODO: Register message: Hi, I wanted to get in touch with you. How are you?
Your message has been registered for sending.


> Finished chain.


{'question': "Email, 'Hi how are you, I wanted to get in touch with you'",
 'output': 'Your message has been registered for sending.'}

In [13]:
AGENT.invoke(
    {
        "question": "Tell me about his passion"
    }
)



> Entering new AgentExecutor chain...
Nikhil’s passion lies in the realm of Artificial Intelligence, particularly focusing on Machine Learning, Deep Learning, Reinforcement Learning, Autonomous Vehicles, and Robotics. He blends creativity with cutting-edge expertise to shape the future of AI. Nikhil excels in crafting advanced ML models using frameworks like PyTorch and TensorFlow, with a dedication to transforming his unique ideologies into tangible products that contribute to impactful projects.

> Finished chain.


{'question': 'Tell me about his passion',
 'output': 'Nikhil’s passion lies in the realm of Artificial Intelligence, particularly focusing on Machine Learning, Deep Learning, Reinforcement Learning, Autonomous Vehicles, and Robotics. He blends creativity with cutting-edge expertise to shape the future of AI. Nikhil excels in crafting advanced ML models using frameworks like PyTorch and TensorFlow, with a dedication to transforming his unique ideologies into tangible products that contribute to impactful projects.'}

In [14]:
AGENT.invoke(
    {
        "question": "Message, 'Can we get in touch soon, this monday'"
    }
)



> Entering new AgentExecutor chain...

Invoking: `register_send_email` with `{'message': 'Can we get in touch soon, this Monday?'}`


TODO: Register message: Can we get in touch soon, this Monday?
Your message has been registered for sending.


> Finished chain.


{'question': "Message, 'Can we get in touch soon, this monday'",
 'output': 'Your message has been registered for sending.'}